<a href="https://colab.research.google.com/github/hyunholee26/Naver-VS-Kakao/blob/main/notebooks/news_keyword_visualization_Naver_VS_Kakao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 구글 드라이브 연결
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [88]:
import os
import glob
import pandas as pd
import numpy as np
import string  

import re

# install packages
!pip install -U -q scattertext
import scattertext as st

import spacy
from IPython.core.display import display, HTML
from scattertext import CorpusFromPandas, produce_scattertext_explorer

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot 
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

In [4]:
# set file path
path = '/gdrive/My Drive/data/naver_vs_kakao/'
  
# read file list
files = glob.glob(path +'*.xlsx')
files.sort()
print(files)

['/gdrive/My Drive/data/naver_vs_kakao/NewsResult_20211015-20220115_kakao.xlsx', '/gdrive/My Drive/data/naver_vs_kakao/NewsResult_20211015-20220115_naver.xlsx']


In [20]:
# read files
df_news_kakao = pd.read_excel('/gdrive/My Drive/data/naver_vs_kakao/NewsResult_20211015-20220115_kakao.xlsx')
df_news_naver = pd.read_excel('/gdrive/My Drive/data/naver_vs_kakao/NewsResult_20211015-20220115_naver.xlsx')

# trim columns
df_news_kakao = (df_news_kakao.loc[df_news_kakao['분석제외 여부'].isnull()])[['키워드']]
df_news_naver = (df_news_naver.loc[df_news_naver['분석제외 여부'].isnull()])[['키워드']]

# set news label
df_news_kakao['Corp'] = 'Kakao'
df_news_naver['Corp'] = 'Naver'

In [22]:
df_news = df_news_kakao.append(df_news_naver).reset_index(drop=True)
df_news

,키워드,Corp
0,"연말,벤처,스타트업,행사,봇물,스타트업,혁신,대한민국,미래,경제,분야,의미,변화,주...",Kakao
1,"멸공,먹튀,논란,여기저기,방영덕,오너,리스크,디테일,방영덕,디테일,본질,표현,자유,...",Kakao
2,"미술시장,발견,신대륙,열광,열광,예술,논란,진행,심층,기획,NFT,아트,1년,NFT...",Kakao
3,"금리,인상,긴축,유동,위축,가치주,시대,증시,가치주,성장주,주도,다툼,시작,코로나1...",Kakao
4,"카카오페이,먹튀,사태,경영,김범수,자율,몸집,엇박자,결과,교수,위정현,중앙대,카카오...",Kakao
...,...,...
15666,"수도,메가시티,수도,인구,전국,인구,50%,생산,혁신인력,집중도,지방,청년들,서울,...",Naver
15667,"소원성취,바위,방면,경상북도,안동,단촌면,소재지,단촌면,관통,화엄종,시조,의상,대사...",Naver
15668,"메타버스ETF,상승,인터넷,게임,종목,구성,부진,투자,증시,대안,메타버스,강세,한국...",Naver
15669,"대교그룹,그린웍스,인수,검토,YG,엔터,자회사,엑스,골프,운영,교육,대교그룹,YG엔...",Naver


In [59]:
sentences = df_news['키워드'].tolist()
print('Number of news : ' + str(len(sentences)))
  
# replace comma to blank
trantab = str.maketrans(",", " ")
sentences = [str(sent).translate(trantab) for sent in sentences]

# remove stopwords
sentences = [re.sub('[^ㄱ-힗]', ' ',str(sent)) for sent in sentences]

sentences_trim = list()
for sent in sentences:
  temp = [word for word in sent.split() if (len(str(word)) > 1)]
  sentences_trim.append(" ".join(temp))

sentences_trim[0:10]

Number of news : 15671


['연말 벤처 스타트업 행사 봇물 스타트업 혁신 대한민국 미래 경제 분야 의미 변화 주도 경제 스타트업 소식 스타트업 전달 소식 투자 유치 업무협약 출시 주제 망라 브랜디 거래액 돌파 코앞 사진 브랜디 브랜디 기준 거래액 돌파 연간 거래액 달성 전망 브랜디 버티컬 커머스 집중 앱스 전략 고성장 세대 여성 패션 쇼핑앱 브랜디 하이버 남자 쇼핑앱 엄마들 육아앱 마미 브랜디 다운로드 누적 거래액 플랫폼별 판매자 증가 거래액 성장 활성 판매자 신규 판매자 대비 전년 동기 증가 판매자 헬피 크리에이터 활동 판매자 전년 대비 가량 헬피 마켓 오픈 서비스 운영 지원 헬피 크리에이터 판매 상품 브랜디 하루배송 서비스 주문 플랫폼 물류 센터 인프라 글로벌 시장 진출 거래액 상승 탄력 서비스 베타 일본 비즈니스 성장 전략 강화 경쟁력 계획 대표이사 브랜디 서정민 대표 이사 플랫폼 풀필먼트 동대문 특화 고객경험 브랜디 유니크 경쟁력 시장 차별화 입지 구축 글로벌 기업 도약 성장 기틀 어메이징브루잉컴퍼니 카카오인베스트먼트 사진 투자 유치 어메이징브루잉컴퍼니 어메이징브루잉컴퍼니 수제 맥주 스타트업 카카오 자회사 투자 전문 카카오인베스트먼트 규모 투자금 유치 어메이징 매출 코로나 상황 전년 대비 가량 오뚜기 협업 진라거 흥행 주효 지역 특산주 맥주 분야 적극 진출 예정 지역 특산주 맥주 온라인 판매 가능 성장 이천 브루어리 건설 브루어리 완공 어메이징 생산량 수준 규모 생산력 확보 생산력 증가세 수출 수요 부응 복안 카카오인베스트먼트 관계자 어메이징 다양성 품질 수제맥주 가치 업체 경쟁업체 인지도 마케팅 투자 한국 대표 수제 맥주 업체 성장 확신 투자 단행 대표 어메이징 김태경 어메이징 금번 투자 본격적 브랜딩 강화 계획 어메이징 수제 맥주 업계 도약 오더플러스 광고모델 발탁 식자재 비교 주문 플랫폼 오더 플러스 광고 모델 조보아 발탁 신규 공개 오더플러스 긍정적 이미지 외식 자영업자 친숙 모습 어필 모델 조보아 기용 조보아 골목식당 프로그램 골목 식당 모습 시청자들 외식 자영업자 반

In [60]:
df_news['키워드'] = sentences_trim
df_news

,키워드,Corp,parsed
0,연말 벤처 스타트업 행사 봇물 스타트업 혁신 대한민국 미래 경제 분야 의미 변화 주...,Kakao,"(연말,벤처,스타트업,행사,봇물,스타트업,혁신,대한민국,미래,경제,분야,의미,변화,..."
1,멸공 먹튀 논란 여기저기 방영덕 오너 리스크 디테일 방영덕 디테일 본질 표현 자유 ...,Kakao,"(멸공,먹튀,논란,여기저기,방영덕,오너,리스크,디테일,방영덕,디테일,본질,표현,자유..."
2,미술시장 발견 신대륙 열광 열광 예술 논란 진행 심층 기획 아트 불가능 토큰 이젠 ...,Kakao,"(미술시장,발견,신대륙,열광,열광,예술,논란,진행,심층,기획,NFT,아트,1년,NF..."
3,금리 인상 긴축 유동 위축 가치주 시대 증시 가치주 성장주 주도 다툼 시작 코로나 ...,Kakao,"(금리,인상,긴축,유동,위축,가치주,시대,증시,가치주,성장주,주도,다툼,시작,코로나..."
4,카카오페이 먹튀 사태 경영 김범수 자율 몸집 엇박자 결과 교수 위정현 중앙대 카카오...,Kakao,"(카카오페이,먹튀,사태,경영,김범수,자율,몸집,엇박자,결과,교수,위정현,중앙대,카카..."
...,...,...,...
15666,수도 메가시티 수도 인구 전국 인구 생산 혁신인력 집중도 지방 청년들 서울 수도권 ...,Naver,"(수도,메가시티,수도,인구,전국,인구,50%,생산,혁신인력,집중도,지방,청년들,서울..."
15667,소원성취 바위 방면 경상북도 안동 단촌면 소재지 단촌면 관통 화엄종 시조 의상 대사...,Naver,"(소원성취,바위,방면,경상북도,안동,단촌면,소재지,단촌면,관통,화엄종,시조,의상,대..."
15668,메타버스 상승 인터넷 게임 종목 구성 부진 투자 증시 대안 메타버스 강세 한국거래소...,Naver,"(메타버스ETF,상승,인터넷,게임,종목,구성,부진,투자,증시,대안,메타버스,강세,한..."
15669,대교그룹 그린웍스 인수 검토 엔터 자회사 엑스 골프 운영 교육 대교그룹 엔터테인먼트...,Naver,"(대교그룹,그린웍스,인수,검토,YG,엔터,자회사,엑스,골프,운영,교육,대교그룹,YG..."


In [64]:
nlp = spacy.load('en')
df_news['parsed'] = df_news['키워드'].apply(nlp)

In [65]:
df_news

,키워드,Corp,parsed
0,연말 벤처 스타트업 행사 봇물 스타트업 혁신 대한민국 미래 경제 분야 의미 변화 주...,Kakao,"(연말, 벤처, 스타트업, 행사, 봇물, 스타트업, 혁신, 대한민국, 미래, 경제,..."
1,멸공 먹튀 논란 여기저기 방영덕 오너 리스크 디테일 방영덕 디테일 본질 표현 자유 ...,Kakao,"(멸공, 먹튀, 논란, 여기저기, 방영덕, 오너, 리스크, 디테일, 방영덕, 디테일..."
2,미술시장 발견 신대륙 열광 열광 예술 논란 진행 심층 기획 아트 불가능 토큰 이젠 ...,Kakao,"(미술시장, 발견, 신대륙, 열광, 열광, 예술, 논란, 진행, 심층, 기획, 아트..."
3,금리 인상 긴축 유동 위축 가치주 시대 증시 가치주 성장주 주도 다툼 시작 코로나 ...,Kakao,"(금리, 인상, 긴축, 유동, 위축, 가치주, 시대, 증시, 가치주, 성장주, 주도..."
4,카카오페이 먹튀 사태 경영 김범수 자율 몸집 엇박자 결과 교수 위정현 중앙대 카카오...,Kakao,"(카카오페이, 먹튀, 사태, 경영, 김범수, 자율, 몸집, 엇박자, 결과, 교수, ..."
...,...,...,...
15666,수도 메가시티 수도 인구 전국 인구 생산 혁신인력 집중도 지방 청년들 서울 수도권 ...,Naver,"(수도, 메가시티, 수도, 인구, 전국, 인구, 생산, 혁신인력, 집중도, 지방, ..."
15667,소원성취 바위 방면 경상북도 안동 단촌면 소재지 단촌면 관통 화엄종 시조 의상 대사...,Naver,"(소원성취, 바위, 방면, 경상북도, 안동, 단촌면, 소재지, 단촌면, 관통, 화엄..."
15668,메타버스 상승 인터넷 게임 종목 구성 부진 투자 증시 대안 메타버스 강세 한국거래소...,Naver,"(메타버스, 상승, 인터넷, 게임, 종목, 구성, 부진, 투자, 증시, 대안, 메타..."
15669,대교그룹 그린웍스 인수 검토 엔터 자회사 엑스 골프 운영 교육 대교그룹 엔터테인먼트...,Naver,"(대교그룹, 그린웍스, 인수, 검토, 엔터, 자회사, 엑스, 골프, 운영, 교육, ..."


In [66]:
st_df = st.CorpusFromParsedDocuments(df_news, category_col='Corp', parsed_col='parsed').build()

In [73]:
html = produce_scattertext_explorer(st_df,
                                    category='Naver',
                                    category_name='Naver',
                                    not_category_name='Kakao',
                                    minimum_term_frequency=500,
                                    width_in_pixels=1000,
                                    max_snippets = 0,
                                    max_docs_per_category = 0,
                                    metadata=df_news['Corp'])
open(path + 'scatter_text_500.html', 'wb').write(html.encode('utf-8'))

/usr/local/lib/python3.7/dist-packages/scattertext/termscoring/ScaledFScore.py:296: RuntimeWarning: invalid value encountered in true_divide
  recall = cat_word_counts * 1. / cat_word_counts.sum()


707054

In [74]:
# 아래 코드가 시각화 되지 않아, 이미지를 첨부하였습니다.
f = open(path + 'scatter_text_500.html', 'r', encoding="utf-8")
html = f.read()
display(HTML(html))

In [112]:
# 연관단어 
def coocc_word(sentences, keyword, window = 3, num = 10):
  coocc = dict()
  rt_str = ""
  for sent in sentences:
    word_list = sent.split(' ')
    length = len(word_list)
    for i,word in enumerate(word_list):
      if word == keyword:
        for j in range(max(i-window,0), min(i+window+1,length)):
          if word_list[j] in coocc:
            coocc[word_list[j]] += 1
          else:
            coocc[word_list[j]] = 1
          #print(coocc)
  df_occ = pd.Series(coocc, name='occ').sort_values(ascending=False)
  return df_occ

def coocc_word_vis(df_news, category, keyword, num_of_result = 11):
  df_co = coocc_word(df_news[df_news['Corp'] == category]['키워드'].to_list(), keyword)[1:num_of_result]
  df_co.iplot(kind = 'bar', color = 'blue', title  = '최근 3개월 ' + category + '관련 뉴스 내 [' + keyword + '] 연관단어 Top 10', xTitle = '연관단어', yTitle = '등장횟수')

# for pandas iplot
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))

In [113]:
configure_plotly_browser_state()
coocc_word_vis(df_news, 'Naver', '공연')

In [115]:
configure_plotly_browser_state()
coocc_word_vis(df_news, 'Naver', '축제')

In [116]:
configure_plotly_browser_state()
coocc_word_vis(df_news, 'Naver', '제페토')

In [117]:
configure_plotly_browser_state()
coocc_word_vis(df_news, 'Naver', '티빙')

In [118]:
configure_plotly_browser_state()
coocc_word_vis(df_news, 'Naver', '네이버클라우드')

In [97]:
configure_plotly_browser_state()
coocc_word_vis(df_news, 'Naver', '플랫폼')

In [98]:
configure_plotly_browser_state()
coocc_word_vis(df_news, 'Kakao', '플랫폼')

In [99]:
configure_plotly_browser_state()
coocc_word_vis(df_news, 'Kakao', '택시')

In [100]:
configure_plotly_browser_state()
coocc_word_vis(df_news, 'Kakao', '카카오모빌리티')

In [101]:
configure_plotly_browser_state()
coocc_word_vis(df_news, 'Kakao', '이모티콘')

In [103]:
configure_plotly_browser_state()
coocc_word_vis(df_news, 'Kakao', '매각')